In [2]:
import random
import numpy as np

my_seed = 1337
random.seed(my_seed)
np.random.seed(my_seed)
import pandas as pd
import numpy as np
from typing import *
from IPython.display import display, HTML, Markdown

import warnings
warnings.filterwarnings('ignore')

### 数据读取与合并

In [3]:
def name_to_id():
    movie_data_columns = [
    'type_douban', 'actor', 'region', 'director', 'characteristic',
    'score', 'moviename']
    movie_data = pd.read_csv(data_dir + './dataset1/movie.csv')
    movie_data.columns = movie_data_columns
    movie_unique = movie_data['moviename'].unique()
    movie_name_to_uninque_index = dict()
    movie_index_to_uninque_name = dict()
    for  i,j in enumerate(movie_unique):
        if i <=22971:
            movie_name_to_uninque_index[j] = i
            movie_index_to_uninque_name[i] = j
        elif i > 22971:
            print('error')
    return movie_name_to_uninque_index,movie_index_to_uninque_name

def load_movies_dataset():
    movie_data_columns = [
    'type_douban', 'actor', 'region', 'director', 'characteristic',
    'score_douban', 'moviename']
    movie_data = pd.read_csv(data_dir + './dataset1/movie.csv')
    print('movie_data\n',movie_data.shape)
    movie_data.columns = movie_data_columns    
    movie_unique = movie_data['moviename'].unique()   
    dict_name_to_id,dict_id_to_name = name_to_id( )    
    movie_data['movie_id'] = movie_data['moviename'].map(lambda x:dict_name_to_id[x])
    
    return movie_data 

def load_user_and_ratings() :
    user_data_columns = ['score_user','user_name','comment_time','user_id','moviename','type_user']
    user_data = pd.read_csv(data_dir + './dataset1/user.csv')
    print('user_data\n',user_data.shape)
    user_data.columns = user_data_columns
    user_data['comment_time'] = pd.to_datetime(user_data['comment_time'])
    dict_name_to_id,dict_id_to_name = name_to_id()   
    user_data['movie_id'] = user_data['moviename'].map(lambda x:dict_name_to_id[x] if x in dict_name_to_id.keys() else '-1')
    
    return user_data 


def load_movielens() :  
    user_data = load_user_and_ratings()
    user_data['user_id'] = user_data['user_id'].map(lambda k: f"User {k}")
    movies_data = load_movies_dataset()
    ratings_and_movies  = pd.merge( user_data, movies_data ,on=['moviename','movie_id'])
    ratings_and_movies['rating'] = ratings_and_movies['score_user']
    del ratings_and_movies['score_user']
    return ratings_and_movies.sample(frac=1).reset_index(drop=True)


data_dir = 'D:/python/Jupyter_Last_project/dataset/'
all_data_df = load_movielens()
print('all_data\n',all_data_df.shape)
print('data info',all_data_df.info())
print('data head\n',all_data_df.head())

user_data
 (188843, 6)
movie_data
 (89524, 7)
all_data
 (741343, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741343 entries, 0 to 741342
Data columns (total 13 columns):
user_name         741343 non-null object
comment_time      741343 non-null datetime64[ns]
user_id           741343 non-null object
moviename         741343 non-null object
type_user         741343 non-null object
movie_id          741343 non-null object
type_douban       741343 non-null object
actor             741343 non-null object
region            741343 non-null object
director          741343 non-null object
characteristic    741343 non-null object
score_douban      741343 non-null float64
rating            741343 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(10)
memory usage: 73.5+ MB
data info None
data head
     user_name        comment_time     user_id   moviename type_user movie_id  \
0  motivation 2018-01-28 15:08:21  User 23502      你好，陌生人        爱情    14912   
1         

In [4]:
#用户的评分是 2 4 6 8 10

print(all_data_df.rating.value_counts())
#豆瓣的评分范围是7.3~10  暂不清楚豆瓣评分机制
print(all_data_df.score_douban.value_counts())

8     295400
4     198297
10    195029
2      52570
6         47
Name: rating, dtype: int64
7.3    29551
7.5    28607
7.6    26888
7.4    26036
7.7    26033
       ...  
2.3      391
2.2      365
2.1      125
9.7      115
9.8        9
Name: score_douban, Length: 78, dtype: int64


Exploring the  dataset 
Douban dataset
user_data 用户评论数据
 (188843, 6)
movie_data 电影数据 
 (89524, 7)
all_data 拼接后的数据
 (741343, 13)
  
 unique len 不重复电影个数
 21722

Training a SVD using Surprise in 4 simple steps

In [5]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

# Step 1: create a Reader.
# A reader tells our SVD what the lower and upper bound of our ratings is.
# MovieLens ratings are from 1 to 5
reader = Reader(rating_scale=(1, 10))
# Step 2: create a new Dataset instance with a DataFrame and the reader
# The DataFrame needs to have 3 columns in this specific order: [user_id, product_id, rating]
data = Dataset.load_from_df(all_data_df[['user_id','moviename','rating']], reader)
# Step 3: keep 25% of your trainset for testing
trainset, testset = train_test_split(data, test_size=.25)
# Step 4: train a new SVD with 100 latent features (number was chosen arbitrarily)
model = SVD(n_factors=500)
model.fit(trainset)
model.qi /= np.linalg.norm(model.qi, ord=2, axis=1).reshape(-1, 1)


In [9]:
print(trainset.n_users) # 
print(trainset.n_items) # 

13545
21713


Inspecting our Product Matrix 
Surprise SVD stores the product matrix under the model.qi attribute.

In [10]:
print('model.qi.shape\n',model.qi.shape)

model.qi.shape
 (21713, 500)


Mapping every vector back to it's movie 
Every row is mapped to a movie. How do we map every movie to it's vector?

In [11]:
#返回的结果是 movie name 和内部的id
def display(df ):
    print('in display')
    #print('list item_to_row_idx.items \n',list(item_to_row_idx.items()))
    #[('怪物大乱斗', 0), ('我想戴上戒指', 1), ('乌云背后的幸福线', 2), ('法官老爹', 3), ('千年追凶', 4), ('日出日落', 5)
    item_to_row_idx_df = pd.DataFrame(
        list(item_to_row_idx.items()),
        columns=['Movie name', 'model.qi row idx'],
    ).set_index('Movie name')
    return item_to_row_idx_df.head(5)

#print('model.trainset._raw2inner_id_items\n',model.trainset._raw2inner_id_items)
#{'怪物大乱斗': 0, '我想戴上戒指': 1, '乌云背后的幸福线': 2, '法官老爹': 3, '千年追凶': 4, '日出日落': 5,
item_to_row_idx  = model.trainset._raw2inner_id_items

# `display()` is a utility function to make `item_to_row_idx` more readable
display(item_to_row_idx)

in display


,model.qi row idx
Movie name,
天生一对,0
金猴降妖,1
足球流氓2,2
冲线,3
伊丽莎白女王,4


Identifying One Movie

In [12]:
#通过电影名得到内部id
toy_story_row_idx  = item_to_row_idx['千年追凶']
#得到内部id 对应的向量
print(model.qi[toy_story_row_idx])
print(f"Every product has {model.qi[toy_story_row_idx].shape[0]} features")

[ 0.06708249 -0.03359497  0.01241809 -0.06068376 -0.04465307 -0.00852593
 -0.00835702 -0.03900419  0.07558425 -0.01974827  0.01298504 -0.01880654
  0.00223658  0.03467793  0.02046255 -0.02462105  0.02018613 -0.03683823
  0.00641584 -0.04048825  0.07525549  0.0015886  -0.0712667   0.00647445
  0.00885059  0.04188474 -0.11382149  0.07007796 -0.021578   -0.04413305
  0.08679129  0.00405458 -0.14782927 -0.00950791  0.01017926  0.01348034
 -0.04427064  0.03446703  0.02494388  0.04166583 -0.03141376  0.00633216
 -0.00595752 -0.00556378 -0.03061609 -0.06447097 -0.01458042  0.00557085
 -0.03148697  0.02210945 -0.00084704  0.04556318 -0.00595579  0.03467261
 -0.01197627 -0.03121101 -0.00035471  0.03439907 -0.00386581  0.00283253
 -0.00918372 -0.06795416  0.03975184 -0.11276918 -0.01508408 -0.01560972
  0.04678589 -0.03293708 -0.01844393 -0.00923351 -0.02008604 -0.06874187
  0.0063792   0.05083313  0.1282675   0.01227655  0.0404464  -0.04007536
  0.01786248  0.02603841 -0.00717236 -0.00325517  0

Recommendations via Product based CF: Finding similarity between vectors 
2 products are "similar" when the cosine distance is close to 0
#一个向量空间中两个向量夹角间的余弦值作为衡量两个个体之间差异的大小，
#余弦值接近1，夹角趋于0，表明两个向量越相似，
#余弦值接近于0，夹角趋于90度，表明两个向量越不相似。
#scipy.spatial.distance.cosine(u，v) 中
#余弦值接近0，夹角趋于0，表明两个向量越相似，
#余弦值接近于1，夹角趋于90度，表明两个向量越不相似。
#u和v之间的余弦距离定义为：1 - u .v / \\u\\ \\v\\

### 通过矩阵重构进行推荐

#### Recommendations via Matrix Reconstruction

Use cases:
Predict a score between any combination of user and a product

Recommendations via Matrix Reconstruction: Using the predict() API inside of Surprise
    
Computes the rating prediction for given user and movie with model.predict(). 
Pick a random user and movie, and calculate the score between them


In [13]:
# Refresher: ratings data-frame.
print(all_data_df.columns)
a1 = all_data_df[['user_id','moviename','rating']]
a1.head(10)

Index(['user_name', 'comment_time', 'user_id', 'moviename', 'type_user',
       'movie_id', 'type_douban', 'actor', 'region', 'director',
       'characteristic', 'score_douban', 'rating'],
      dtype='object')


,user_id,moviename,rating
0,User 23502,你好，陌生人,10
1,User 26682,死神有约：死后的生活,8
2,User 3249,精灵旅社2,8
3,User 4874,盲者之国,2
4,User 1650,安娜·卡列尼娜,8
5,User 6647,铁面无私,10
6,User 4372,史酷比与国王的精灵,4
7,User 26935,梁祝,2
8,User 17542,囚犯,8
9,User 517,风斗士,2


In [14]:
a_user = "User 15668"
a_product = "梁祝"
model.predict(a_user, a_product)

Prediction(uid='User 15668', iid='梁祝', r_ui=None, est=6.625245834510631, details={'was_impossible': False})

### 通过基础CF 进行推荐：查找向量之间的相似度

Item-based collaborative filtering
Recommendations via Item Similarity: Finding similarity between vectors
2 products are "similar" when the cosine distance is close to 0

In [15]:
from scipy.spatial.distance import cosine

#通过电影名得到内部id
#返回值：内部id 对应的向量
def get_vector_by_movie_title(movie_title ,trained_model )  :
    """Returns the latent features of a movie in the form of a numpy array"""
    movie_row_idx = trained_model.trainset._raw2inner_id_items[movie_title]
    return trained_model.qi[movie_row_idx]


def cosine_distance(vector_a , vector_b ) :
    """Returns a float indicating the similarity between two vectors"""
    return cosine(vector_a, vector_b)

In [16]:
from scipy.spatial.distance import cosine as cosine_distance

# Fetch indices for Toy Story and Wizard of Oz
djs_idx = model.trainset._raw2inner_id_items['独角兽']
hy_idx = model.trainset._raw2inner_id_items['后裔']
westar_idx = model.trainset._raw2inner_id_items['我们的追星之路']

# Get vectors for both movies
starwars_vector = model.qi[djs_idx]
return_of_jedi_vector = model.qi[hy_idx]
aladdin_vector = model.qi[westar_idx]
# Distance 1
print(cosine_distance(starwars_vector, return_of_jedi_vector))
# Distance 2
print(cosine_distance(starwars_vector, aladdin_vector))

1.0601859743377933
0.9954850171201605


### Finding similar movies by ranking

In [17]:
def display_similarity(similarity_table):
    similarity_table = pd.DataFrame(
        similarity_table,
        columns=['vector cosine distance', 'movie title']
    ).sort_values('vector cosine distance', ascending=True)
    return similarity_table.iloc[:6]

In [18]:
def get_top_similarities(movie_title , model ):
    """Returns the top 5 most similar movies to a specified movie
    
    This function iterates over every possible movie in MovieLens and calculates
    distance between `movie_title` vector and that movie's vector.
    """
    
    # Get the first movie vector
    #通过电影名得到内部id
    #返回值：内部id 对应的向量
    movie_vector  = get_vector_by_movie_title(movie_title, model)
    similarity_table = []
    
    # Iterate over every possible movie and calculate similarity
    #print('model.trainset._raw2inner_id_items\n',model.trainset._raw2inner_id_items)
    #dict '怪物大乱斗': 0, '我想戴上戒指': 1, '乌云背后的幸福线': 2, '法官老爹': 3, '千年追凶': 4, 
    #key 电影名  value  内部id（inner id）
    for other_movie_title in model.trainset._raw2inner_id_items.keys():
        other_movie_vector = get_vector_by_movie_title(other_movie_title, model)   
        # Get the second movie vector, and calculate distance
        similarity_score = cosine_distance(other_movie_vector, movie_vector)
        similarity_table.append((similarity_score, other_movie_title))

    #列表的每个元素是元组 （相似度，电影名）
    print('similarity_table len\n',len(similarity_table))
    
    # sort movies by ascending similarity
    return display_similarity(sorted(similarity_table))

In [19]:
get_top_similarities('王牌对王牌', model)

similarity_table len
 21713


,vector cosine distance,movie title
0,0.000000,王牌对王牌
1,0.818190,恐怖人生
2,0.828107,猫的奴隶
3,0.835681,无法阻挡的婚姻
4,0.838362,赛文奥特曼 EVOLUTION EPISODE：1 黑暗一面
5,0.838501,日以作夜
